In [1]:
import nltk
import threading
import queue
import feedparser
import uuid

In [2]:
threads = []
queues = [queue.Queue(), queue.Queue()]

In [8]:
def extractWords():
    url = 'http://timesofindia.indiatimes.com/rssfeeds/1081479906.cms'
    feed = feedparser.parse(url)
    for entry in feed['entries'][:5]:
        text = entry['title']
        if 'ex' in text:
            continue
        words = nltk.word_tokenize(text)
        data = {'uuid':uuid.uuid4(),'input':words}
        queues[0].put(data,True)
        print(">>{} : {}".format(data['uuid'],text))
        
def extractPOS():
    while True:
        if queues[0].empty():
            break
        else:
            data = queues[0].get()
            words = data['input']
            postags = nltk.pos_tag(words)
            queues[0].task_done()
            queues[1].put({'uuid': data['uuid'], 'input':postags},True)

def extractNE():
    while True:
        if queues[1].empty():
            break
        else:
            data = queues[1].get()
            postags = data['input']
            queues[1].task_done()
            chunks = nltk.ne_chunk(postags, binary=False)
            print(" << {} :".format(data['uuid']), end='')
            for path in chunks:
                try:
                    label = path.label()
                    print(path, end = ', ')
                except:
                    pass
                print()

def runProgram():
    e = threading.Thread(target=extractWords())
    e.start()
    threads.append(e)
    
    p = threading.Thread(target=extractPOS())
    p.start()
    threads.append(p)
    
    n = threading.Thread(target=extractNE())
    n.start()
    threads.append(n)
    
    queues[0].join()
    queues[1].join()
    
    for t in threads:
        t.join()
        
if __name__ == '__main__':
    runProgram()

>>a015b47e-48aa-452f-a73c-b3f3676fd765 : SRK & Arbaaz's dad share the same birthday
>>03d9f59b-c378-40a7-9e3e-927e41b24774 : Twitter shares memes on Anushka-Virat
>>401e3563-8693-4dc8-ba29-25aa3f94f51f : Fans of Alia Bhatt discover her lookalike
>>9d08ac71-c9a5-4d13-919d-c7ec522b21ce : Newlyweds who will celebrate their 1st Diwali
>>533f1574-7346-41e6-8027-b7586e6d1754 : Hansal Mehta: SRK is my superstar forever
 << 2ebedf8e-650a-4244-bd2c-ca0b2ab066df :(ORGANIZATION SRK/NNP), 

(PERSON Arbaaz/NNP), 






 << fa881a08-9179-44ff-918f-234496f1861a :




 << 2eebdfaa-4b00-45c5-9916-096b7359dd2c :

(PERSON Alia/NNP Bhatt/NNP), 



 << 939b8be3-b396-498d-b9f2-2dbe6c18d4b3 :






 << e1eca8af-6a53-4906-a6e1-8582555fedf7 :(ORGANIZATION SRK/NNP), 

(PERSON Arbaaz/NNP), 






 << 2357a640-df6b-4d8a-92e2-62e538d9ad08 :




 << 9bfa5a93-3c04-47c5-be0a-e97c28f51e94 :

(PERSON Alia/NNP Bhatt/NNP), 



 << c655f243-aebb-4ce3-9357-4690863522b0 :






 << d11efb01-1cc9-44e5-92f8-098bf2909c0c :(PER